# Overview
FIXME...

In [1]:
import os
from langchain_neo4j import Neo4jGraph

kg = Neo4jGraph(
    url=os.getenv("NEO4J_URI"), 
    username=os.getenv("NEO4J_USERNAME"), 
    password=os.getenv("NEO4J_PASSWORD"),
    database=os.getenv("NEO4J_DATABASE")
)

In [ ]:
config = {
    "llm": {
        "provider": "huggingface",    # options: "ollama", "openai", "huggingface", etc.
        'model_id': 'deepseek-ai/DeepSeek-R1-Distill-Qwen-7B',
        "max_seq_len": 16384,
        "num_predict": 16384,
        "temperature": 0.5,
        "api_key": "YOUR_API_KEY_IF_NEEDED"  # only used for some providers
    },
    'database': {
        'uri': os.getenv("NEO4J_URI"),
        'username': os.getenv("NEO4J_USERNAME"),
        'password': os.getenv("NEO4J_PASSWORD"),
        'database': os.getenv("NEO4J_DATABASE")
    },
    "embedding": {
        "provider": "huggingface",
        "model_id": 'sentence-transformers/multi-qa-mpnet-base-dot-v1'
    },
    'rag': {
        'index_name': 'paper_chunks',
        'embedding_node_property': 'textEmbedding',
        'text_node_property': 'text'
    }
}

In [3]:
import sys
sys.path.append("/app/src")  # Add to Python's search path

# Now you can import modules
from rag import rag_utils as rag  # Replace with actual module name

llm_adapter = rag.get_llm(config)
emb_adapter = rag.get_embeddings(config)

In [4]:
custom_query = """
MATCH (c:Chunk)
WITH DISTINCT c, vector.similarity.cosine(c.textEmbedding, $embedding) AS score
ORDER BY score DESC LIMIT $k
RETURN c.text AS text, score, {source: c.source, chunkId: c.chunkId} AS metadata
"""

prompt_template = """
You are an expert in generative models. Based on the following context, provide a concise answer to the question. 
If the context lacks the requested information simply state that you do not know.

Context:
{context}

Question:
{question}

Answer:
"""

question = "What are some ways that you can leverage the structure of a latent space to influence generation in a diffusion model?"

rag.query_kg(question, 
    llm_adapter, 
    emb_adapter, 
    kg, 
    config, 
    custom_query, 
    prompt_template, 
    k=10
)

ConnectionError: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download